In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# To ignore warinings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Random forest là gì ?

## Định nghĩa.

Random forest là một phương pháp thống kê mô hình hóa bằng máy (machine learning statistic) dùng để phục vụ các mục đích phân loại, hồi quy và các nhiệm vụ khác bằng cách xây dựng nhiều cây quyết định (Decision tree).

[_Nguồn_](https://jgac.vn/journal/article/view/344#:~:text=Random%20forest%20l%C3%A0%20m%E1%BB%99t%20ph%C6%B0%C6%A1ng,quy%E1%BA%BFt%20%C4%91%E1%BB%8Bnh%20(Decision%20tree).)

Random Forest được ghi nhận hiệu quả hơn so với thuật toán phân loại khác thường được sử dụng vì có khả năng tìm ra thuộc tính nào quan trọng hơn so với những thuộc tính khác.

</br>

![](https://res.cloudinary.com/dyd911kmh/image/upload/v1677239993/image5_c214968fd6.png)

<br>

**Ưu điểm:**

Random forests được coi là một phương pháp chính xác và mạnh mẽ vì số cây quyết định tham gia vào quá trình này. Thuật toán có thể được sử dụng trong cả hai vấn đề phân loại và hồi quy.

<br>

**Nhược điểm:**

Random forests chậm tạo dự đoán bởi vì nó có nhiều cây quyết định. Bất cứ khi nào nó đưa ra dự đoán, tất cả các cây trong rừng phải đưa ra dự đoán cho cùng một đầu vào cho trước và sau đó thực hiện bỏ phiếu trên đó. Toàn bộ quá trình này tốn rất nhiều thời gian.

Mô hình khó hiểu hơn so với cây quyết định, nơi ta có thể dễ dàng đưa ra quyết định bằng cách đi theo đường dẫn trong cây.

## Cách thức hoạt động.

1. Chọn các mẫu ngẫu nhiên từ tập dữ liệu đã cho.

2. Thiết lập cây quyết định cho từng mẫu và nhận kết quả dự đoán từ mỗi quyết định cây.

3. Bỏ phiếu cho mỗi kết quả dự đoán.

4. Chọn kết quả được dự đoán nhiều nhất làm dự đoán cuối cùng.

<br>

![](https://images.viblo.asia/33adb558-67ba-4262-809e-3f8a8348e0c8.png)

## Decision tree.

Decision tree (cây quyết định) là một mô hình học máy dựa trên việc xây dựng cây có cấu trúc phân cấp để đưa ra các quyết định hoặc dự đoán. Cây quyết định được sử dụng chủ yếu trong bài toán phân loại (classification) và dự đoán (prediction).

<br>

Cây quyết định có cấu trúc giống cây với các nút (node) và các nhánh (branch). Các nút trong cây đại diện cho các quyết định hoặc các thuộc tính và các nhánh đại diện cho các kết quả hoặc các lựa chọn khả thi. Từ nút gốc, cây phân nhánh xuống các nút con dựa trên giá trị của thuộc tính được chọn để tối ưu hóa việc phân loại hoặc dự đoán. Quá trình này tiếp tục cho đến khi đạt được các nút lá (leaf node) đại diện cho các kết quả cuối cùng.

<br>

![](https://insidelearningmachines.com/wp-content/uploads/2021/02/tree_diagram.png)

<br>

Trong quá trình xây dựng cây quyết định, thuật toán sẽ sử dụng các tiêu chí như information gain (lợi ích thông tin), Gini index hoặc chi-square để đánh giá và lựa chọn thuộc tính phân nhánh tốt nhất. Các quyết định và dự đoán được thực hiện bằng cách đi xuống cây theo các nhánh tương ứng với các giá trị thuộc tính.

<br>

Trong bài toán này, team sẽ sử dụng information gain, cụ thể là overall entropy để tính toán cho nhanh vì `minimun overall entropy` = `maximun information gain`. Công thức tính như sau:

<br>

$ Overall Entropy = p_1 * Entropy(S_1) + p_2 * Entropy(S_2) + ... + p_n * Entropy(S_n)$

Trong đó:

- Overall Entropy là overall entropy.
- p1, p2, ..., pn là tỷ lệ số lượng mẫu trong các tập dữ liệu con S1, S2, ..., Sn.
- Entropy(S1), Entropy(S2), ..., Entropy(Sn) là entropy của các tập dữ liệu con.

<br>

Công thức tính entropy:

$ Entropy(S) = - ∑ (p_i * log2(p_i)) $

Trong đó:

- Entropy(S) là entropy của tập dữ liệu S.
- $p_i$ là tỷ lệ số lượng mẫu thuộc vào lớp i trong tập dữ liệu S.
- log2 là hàm logarithm cơ số 2.



## Ứng dụng thực tế.

Chủ yếu áp dụng vào việc giải các bài toán liên quan đến phân loại và hồi quy.

<br>

**Ví dụ:** Phân loại bệnh hoặc dự đoán giá cả hàng hóa trong tương lai.

# Hướng phát triển của nhóm.

Áp dụng Numba, CUDA để chạy song song cho thuật toán Random forest nhằm mục đích cài thiện tốc độ trong quá trình sử dụng. Tiết kiệm thời gian huấn luyện và tận dụng tốt tài nguyên máy nhất có thể.

# Nguồn tài liệu.

Nhóm tìm được thuật toán Random Forest từ một [nguồn trên github](https://github.com/harrypnh/random-forest-from-scratch), họ implement thuật toán với 2 mục đích chính: chẩn đoán bệnh ung thư vú và đánh giá ô tô ( Car Evalution ).

<br>

**Tại sao lại chọn từ nguồn này?**

Vì họ xây dựng thuật toán Random Forest theo hàm chứ không xây theo class, điều này khiến cho quá trình làm việc với numba trở nên dễ dàng hơn.


# Dữ liệu đầu vào.

Về phần dataset, có lẽ nhóm sẽ tìm một dataset nào đó đủ lớn và dễ hiểu để training model trước. Việc này sẽ dễ hơn là dùng lại dataset có sẵn vì các kiến thức về xe ô tô và y tế nhóm chưa có nhiều kinh nghiệm nên việc diễn giải sẽ khó hơn.

# Mục tiêu cần đạt được.

xây dựng ra tất cả 4 version:
- V1 - là thuật toán Random Forest chạy tuần tự.
- V2 - là thuật toán Random Forest chạy song song bằng CPU.
- V3 - là thuật toán Random Forest chạy song song bằng GPU.
- V4 - là thuật toán Random Forest chạy song song sử dụng shared memory.

<br>

Ở mức kỳ vọng 100% - nhóm mong muốn có thể xây dựng được thuật toán đến V3 ( GPU ) và tốc độ có sự cải thiện khi chạy với dataset đủ lớn.

<br>

Ở mức kỳ vọng 125% - nhóm mong muốn xây dựng được thuật toán đến V4 và có thể ứng dụng thuật toán của nhóm vào để thử giải quyết một bài toán thực tế nào đó do các thành viên nhóm đề cử ( ví dụ chuẩn đoán bệnh ).


# Utility functions

In [ ]:
# Hàm hỗ trợ phân tách tập train - test

def trainTestSplit(dataFrame, testSize):
    if isinstance(testSize, float):
        testSize = round(testSize * len(dataFrame))
    indices = dataFrame.index.tolist()
    testIndices = random.sample(population = indices, k = testSize)
    dataFrameTest = dataFrame.loc[testIndices]
    dataFrameTrain = dataFrame.drop(testIndices)
    return dataFrameTrain, dataFrameTest

## Khái quát

Để xây dựng thuật toán Random forest ( tuần tự ) thì ở đây, nhóm cần build 2 phần từ là:

1. **DecisionTree:** Xử lý các vấn đề liên quan đến cây quyết định.

2. **RandomForest:** Xử lý các vấn đề liên quan đến random forest.



# 1. Random Forest tuần tự **[ V1 ]**

## Decision Tree

In [ ]:
import numpy
import pandas
import random


'''
Kiểm tra data có thuộc một label duy nhất
'''
def checkPurity(data):
    if len(np.unique(data[:, -1])) == 1:
        return True
    else:
        return False


'''
Xác định label phổ biến nhất còn trong data và trả về label đó

'''
def classifyData(data):
    uniqueClasses, uniqueClassesCounts = np.unique(data[:, -1], return_counts = True)
    return uniqueClasses[uniqueClassesCounts.argmax()]


'''
Xác định cột (thuộc tính) và các giá trị split tiềm năng của cột đó
'''
def getPotentialSplits(data, randomAttributes):
    potentialSplits = {}
    _, columns = data.shape

    # Lấy hết thuộc tính (trừ cột label cuối cùng) gán vào columnsIndices (List gồm index của các thuộc tính)
    columnsIndices = list(range(columns - 1))

    # Nếu có randomAttributes thì không lấy hết
    if randomAttributes != None  and len(randomAttributes) <= len(columnsIndices):
        columnsIndices = randomAttributes
    for column in columnsIndices:
        values = data[:, column]
        # Xét độ unique của mỗi thuộc tính
        uniqueValues = np.unique(values)

        # Nếu thuộc tính chỉ có đúng 1 giá trị thì potentialSplits tại cột (thuộc tính) đó = chính giá trị đó
        if len(uniqueValues) == 1:
            potentialSplits[column] = uniqueValues

        # còn không thì potentialSplits tại cột (thuộc tính) đó sẽ chứa các giá trị trung bình giữa 2 giá trị kế nhau trong mảng uniqueValues
        else:
            potentialSplits[column] = []
            for i in range(len(uniqueValues)):
                if i != 0:
                    currentValue = uniqueValues[i]
                    previousValue = uniqueValues[i - 1]
                    potentialSplits[column].append((currentValue + previousValue) / 2)
    return potentialSplits


'''
# Tách data thành 2 phần dựa vào splitValue
'''
def splitData(data, splitColumn, splitValue):
    # Lấy data của cột thuộc tính đang đc xét để so sánh
    splitColumnValues = data[:, splitColumn]

    # Tách nguyên data (không phải chỉ mỗi data ở cột splitColumnValues) thành hai phần, một phần gồm các giá trị ở cột splitColumnValues <= splitValue và ngược lại
    return data[splitColumnValues <= splitValue], data[splitColumnValues > splitValue]

'''
# Tính entropy của từng phần data
'''
def calculateEntropy(data):
    # Tính entropy của data dựa vào tỷ lệ labels mà data thuộc về
    _, uniqueClassesCounts = np.unique(data[:, -1], return_counts = True)
    probabilities = uniqueClassesCounts / uniqueClassesCounts.sum()
    return sum(probabilities * -np.log2(probabilities))


'''
# Tính tổng entropy
'''
def calculateOverallEntropy(dataBelow, dataAbove):
    # Lấy tỷ lệ số lượng data ở nửa trên so với số lượng data
    pDataBelow = len(dataBelow) / (len(dataBelow) + len(dataAbove))

    # Lấy tỷ lệ số lượng data ở nửa dưới so với số lượng data
    pDataAbove = len(dataAbove) / (len(dataBelow) + len(dataAbove))
    return pDataBelow * calculateEntropy(dataBelow) + pDataAbove * calculateEntropy(dataAbove)


'''
Xác định cột (thuộc tính) và giá trị tiềm năng duy nhất của cột đó (splitPoint có entropy nhỏ nhất)
'''
def determineBestSplit(data, potentialSplits, randomSplits = None):
    overallEntropy = 9999
    bestSplitColumn = 0
    bestSplitValue = 0
    if randomSplits == None:
        # Xét từng key là thuộc tính trong potentialSplits
        for splitColumn in potentialSplits:
            # Xét từng value của từng thuộc tính trong potentialSplits
            for splitValue in potentialSplits[splitColumn]:
                # Tách nguyên data của làm hai phần nhỏ hơn và lớn hơn so với value đang xét
                dataBelow, dataAbove = splitData(data, splitColumn, splitValue)
                # Xét Entropy của 2 phần data đó, nếu nhỏ nhất thì lấy
                currentOverallEntropy = calculateOverallEntropy(dataBelow, dataAbove)
                if currentOverallEntropy <= overallEntropy:
                    overallEntropy = currentOverallEntropy
                    bestSplitColumn = splitColumn
                    bestSplitValue = splitValue
    else:
      # Tương tự như phần trên nhưng nó chọn random thuộc tính sau đó chọn random value để tính entropy
        for i in range(randomSplits):
            randomSplitColumn = random.choice(list(potentialSplits))
            randomSplitValue = random.choice(potentialSplits[randomSplitColumn])
            dataBelow, dataAbove = splitData(data, randomSplitColumn, randomSplitValue)
            currentOverallEntropy = calculateOverallEntropy(dataBelow, dataAbove)
            if currentOverallEntropy <= overallEntropy:
                overallEntropy = currentOverallEntropy
                bestSplitColumn = randomSplitColumn
                bestSplitValue = randomSplitValue
    return bestSplitColumn, bestSplitValue


'''
Xây dựng cây
'''
def buildDecisionTree(dataFrame, currentDepth = 0, minSampleSize = 2, maxDepth = 1000, randomAttributes = None, randomSplits = None):
    # Nếu mới bắt đầu khởi tạo cây
    if currentDepth == 0:
        global COLUMN_HEADERS
        COLUMN_HEADERS = dataFrame.columns
        data = dataFrame.values

        # Nếu randomAttributes = 0, xét hết tất cả thuộc tính còn không thì xét một vài thuộc tính được lấy random ra.
        if randomAttributes != None and randomAttributes <= len(COLUMN_HEADERS) - 1:
            randomAttributes = random.sample(population = list(range(len(COLUMN_HEADERS) - 1)), k = randomAttributes)
        else:
            randomAttributes = None
    else:
        data = dataFrame
    # Xét nếu tất cả dữ liệu đều có label giống nhau || dữ liệu dữ liệu ít hơn số dữ liệu tối thiểu để xét cho một node || độ cao đã đạt tối đa
    if checkPurity(data) or len(data) < minSampleSize or currentDepth == maxDepth:
        # Trả về label cho node cuối cùng đó
        return classifyData(data)

    # Nếu không thì tiếp tục quá trình tạo cây
    else:
        currentDepth += 1
        # Lấy ra potentialSplits là dictionaries gồm key là các thuộc tính, values là các giá trị potential của mỗi thuộc tính
        potentialSplits = getPotentialSplits(data, randomAttributes)


        # Lấy ra cột thuộc tính và giá trị split của thuộc tính mà mang lại giá trị entropy nhỏ nhất
        splitColumn, splitValue = determineBestSplit(data, potentialSplits, randomSplits)

        # split data theo 2 giá trị vừa lấy đc
        dataBelow, dataAbove = splitData(data, splitColumn, splitValue)

        # Nếu tất cả data đều thuộc một label rồi thì lấy label đó cho node
        if len(dataBelow) == 0 or len(dataAbove) == 0:
            return classifyData(data)
        else:
            # Đặt giá trị splitPoint cho thuộc tính đó, phần data nhỏ hơn sẽ qua yes và ngược lại
            question = str(COLUMN_HEADERS[splitColumn]) + " <= " + str(splitValue)
            decisionSubTree = {question: []}
            # Tiếp tục rẽ nhánh bằng đệ quy
            yesAnswer = buildDecisionTree(dataBelow, currentDepth, minSampleSize, maxDepth, randomAttributes, randomSplits)
            noAnswer = buildDecisionTree(dataAbove, currentDepth, minSampleSize, maxDepth, randomAttributes, randomSplits)
            # Nếu 2 nhánh giống nhau thì chỉ rẽ 1 nhánh, khác nhau thì rẽ 2 nhánh
            if yesAnswer == noAnswer:
                decisionSubTree = yesAnswer
            else:
                decisionSubTree[question].append(yesAnswer)
                decisionSubTree[question].append(noAnswer)
            return decisionSubTree


'''
Dùng để xét label cho tập dữ liệu bằng cách đưa mỗi cây trong randomForest
'''
def classifySample(sample, decisionTree):
    # Nếu xét đến label của node lá cuối cùng thì trả về label đó
    if not isinstance(decisionTree, dict):
        return decisionTree
    question = list(decisionTree.keys())[0]
    attribute, value = question.split(" <= ")
    # So sánh với splitValue để mò đường đi tới node chứa label quyết định
    if sample[attribute] <= float(value):
        answer = decisionTree[question][0]
    else:
        answer = decisionTree[question][1]
    return classifySample(sample, answer)


'''
Trả về label được dự đoán của tập data
'''
def decisionTreePredictions(dataFrame, decisionTree):
    # Dự đoán từng label của từng dòng dữ liệu
    predictions = dataFrame.apply(classifySample, axis = 1, args = (decisionTree,))
    return predictions


'''
# So sánh độ chính xác giữa tập đang ktra và tập giá trị thật
'''
def calculateAccuracy(predictedResults, category):
    resultCorrect = predictedResults == category
    return resultCorrect.mean()

## Random forest

In [ ]:
'''
Dùng kỹ thuật boostrap để lấy random một subset của data
'''
def bootstrapSample(dataFrame, bootstrapSize):
    randomIndices = np.random.randint(low = 0, high = len(dataFrame), size = bootstrapSize)
    return dataFrame.iloc[randomIndices]


'''
Xây dựng rừng cây từ các cây decision tree
'''
def createRandomForest(dataFrame, bootstrapSize, randomAttributes, randomSplits, forestSize = 20, treeMaxDepth = 1000):
    forest = []
    for i in range(forestSize):
        # Lấy random một subset của data bằng kỹ thuật boostrap
        bootstrappedDataFrame = bootstrapSample(dataFrame, bootstrapSize)
        # Xây dựng 1 cây quyết định từ subset đó
        decisionTree = buildDecisionTree(bootstrappedDataFrame, maxDepth = treeMaxDepth, randomAttributes = randomAttributes, randomSplits = randomSplits)
        # Thêm cây vào rừng
        forest.append(decisionTree)
    return forest


'''
Dự đoán data bằng cách đưa qua từng cây trong randomForest rồi lấy label theo số đông
'''
def randomForestPredictions(dataFrame, randomForest):
    predictions = {}
    # Đưa dữ liệu qua rừng cây (cụ thể là từng cây)
    for i in range(len(randomForest)):

        column = "decision tree " + str(i)
        predictions[column] = decisionTreePredictions(dataFrame, randomForest[i])
    predictions = pd.DataFrame(predictions)
    # Lấy label có số lần xuất hiện nhiều nhất của từng dòng
    return predictions.mode(axis = 1)[0]

'''
# So sánh độ chính xác giữa tập đang ktra và tập giá trị thật
'''
def calculateAccuracy(predictedResults, category):
    # So sánh label của tập dự đoán và tập real
    resultCorrect = predictedResults == category
    return resultCorrect.mean()

## Sample run

In [ ]:
# Load the dataset
df = pd.read_csv("/content/drive/MyDrive/Bài Tập Về Nhà Môn Cuối/iris.csv")

# label encoder
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()

# Encode labels in column 'species'.
df['species']= label_encoder.fit_transform(df['species'])

%time
randomForest = createRandomForest(df, bootstrapSize = 10, randomAttributes = 10, randomSplits = 50, numTree = 30, maxDepth = 3)
randomForestTestResults = randomForestPredictions(df, randomForest)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.53 µs


In [ ]:
accuracy = calculateAccuracy(randomForestTestResults, df.iloc[:, -1]) * 100

In [ ]:
accuracy

96.0

# 2. Song song hóa sử dụng CPU **[ V2 ]**

## Decision Tree V2.0

In [ ]:
import numpy
import pandas
import random
from numba import njit, prange



def checkPurity(data):
    if len(numpy.unique(data[:, -1])) == 1:
        return True
    else:
        return False



def classifyData(data):
    uniqueClasses, uniqueClassesCounts = numpy.unique(data[:, -1], return_counts = True)
    return uniqueClasses[uniqueClassesCounts.argmax()]



def getPotentialSplits(data, randomAttributes):
    potentialSplits = {}
    _, columns = data.shape
    columnsIndices = list(range(columns - 1))
    if randomAttributes != None  and len(randomAttributes) <= len(columnsIndices):
        columnsIndices = randomAttributes
    for column in columnsIndices:
        values = data[:, column]
        uniqueValues = numpy.unique(values)
        if len(uniqueValues) == 1:
            potentialSplits[column] = uniqueValues
        else:
            potentialSplits[column] = []
            for i in prange(len(uniqueValues)): # // ở đây bằng prange
                if i != 0:
                    currentValue = uniqueValues[i]
                    previousValue = uniqueValues[i - 1]
                    potentialSplits[column].append((currentValue + previousValue) / 2)
    return potentialSplits



def splitData(data, splitColumn, splitValue):
    splitColumnValues = data[:, splitColumn]
    return data[splitColumnValues <= splitValue], data[splitColumnValues > splitValue]


njit(parallel=True)
def calculateEntropy(data):
    _, uniqueClassesCounts = numpy.unique(data[:, -1], return_counts = True)
    probabilities = uniqueClassesCounts / uniqueClassesCounts.sum()

    # // ở đây, thay thế hàm sum có sẵn trong python bằng vòng lặp sử dụng prange
    entropy = 0.0
    for i in prange(len(probabilities)):
        entropy -= probabilities[i] * np.log2(probabilities[i])
    return entropy



def calculateOverallEntropy(dataBelow, dataAbove):
    pDataBelow = len(dataBelow) / (len(dataBelow) + len(dataAbove))
    pDataAbove = len(dataAbove) / (len(dataBelow) + len(dataAbove))
    return pDataBelow * calculateEntropy(dataBelow) + pDataAbove * calculateEntropy(dataAbove)



def determineBestSplit(data, potentialSplits, randomSplits = None):
    overallEntropy = 9999
    bestSplitColumn = 0
    bestSplitValue = 0
    if randomSplits == None:
        for splitColumn in potentialSplits:
            for splitValue in potentialSplits[splitColumn]:
                dataBelow, dataAbove = splitData(data, splitColumn, splitValue)
                currentOverallEntropy = calculateOverallEntropy(dataBelow, dataAbove)
                if currentOverallEntropy <= overallEntropy:
                    overallEntropy = currentOverallEntropy
                    bestSplitColumn = splitColumn
                    bestSplitValue = splitValue
    else:
        for i in prange(randomSplits): # // vòng for này bằng prange
            randomSplitColumn = random.choice(list(potentialSplits))
            randomSplitValue = random.choice(potentialSplits[randomSplitColumn])
            dataBelow, dataAbove = splitData(data, randomSplitColumn, randomSplitValue)
            currentOverallEntropy = calculateOverallEntropy(dataBelow, dataAbove)
            if currentOverallEntropy <= overallEntropy:
                overallEntropy = currentOverallEntropy
                bestSplitColumn = randomSplitColumn
                bestSplitValue = randomSplitValue
    return bestSplitColumn, bestSplitValue



def buildDecisionTree(dataFrame, currentDepth = 0, minSampleSize = 2, maxDepth = 1000, randomAttributes = None, randomSplits = None):
    if currentDepth == 0:
        global COLUMN_HEADERS
        COLUMN_HEADERS = dataFrame.columns
        data = dataFrame.values
        if randomAttributes != None and randomAttributes <= len(COLUMN_HEADERS) - 1:
            randomAttributes = random.sample(population = list(range(len(COLUMN_HEADERS) - 1)), k = randomAttributes)
        else:
            randomAttributes = None
    else:
        data = dataFrame
    if checkPurity(data) or len(data) < minSampleSize or currentDepth == maxDepth:
        return classifyData(data)
    else:
        currentDepth += 1
        potentialSplits = getPotentialSplits(data, randomAttributes)
        splitColumn, splitValue = determineBestSplit(data, potentialSplits, randomSplits)
        dataBelow, dataAbove = splitData(data, splitColumn, splitValue)
        if len(dataBelow) == 0 or len(dataAbove) == 0:
            return classifyData(data)
        else:
            question = str(COLUMN_HEADERS[splitColumn]) + " <= " + str(splitValue)
            decisionSubTree = {question: []}
            yesAnswer = buildDecisionTree(dataBelow, currentDepth, minSampleSize, maxDepth, randomAttributes, randomSplits)
            noAnswer = buildDecisionTree(dataAbove, currentDepth, minSampleSize, maxDepth, randomAttributes, randomSplits)
            if yesAnswer == noAnswer:
                decisionSubTree = yesAnswer
            else:
                decisionSubTree[question].append(yesAnswer)
                decisionSubTree[question].append(noAnswer)
            return decisionSubTree



def classifySample(sample, decisionTree):
    if not isinstance(decisionTree, dict):
        return decisionTree
    question = list(decisionTree.keys())[0]
    attribute, value = question.split(" <= ")
    if sample[attribute] <= float(value):
        answer = decisionTree[question][0]
    else:
        answer = decisionTree[question][1]
    return classifySample(sample, answer)



def decisionTreePredictions(dataFrame, decisionTree):
    predictions = dataFrame.apply(classifySample, axis = 1, args = (decisionTree,))
    return predictions



def calculateAccuracy(predictedResults, category):
    resultCorrect = predictedResults == category
    return resultCorrect.mean()

## Random Forest V2.0

In [ ]:
def trainTestSplit(dataFrame, testSize):
    if isinstance(testSize, float):
        testSize = round(testSize * len(dataFrame))
    indices = dataFrame.index.tolist()
    testIndices = random.sample(population = indices, k = testSize)
    dataFrameTest = dataFrame.loc[testIndices]
    dataFrameTrain = dataFrame.drop(testIndices)
    return dataFrameTrain, dataFrameTest

def bootstrapSample(dataFrame, bootstrapSize):
    randomIndices = numpy.random.randint(low = 0, high = len(dataFrame), size = bootstrapSize)
    return dataFrame.iloc[randomIndices]



def createRandomForest(dataFrame, bootstrapSize, randomAttributes, randomSplits, forestSize = 20, treeMaxDepth = 1000):
    forest = []
    for i in prange(forestSize): # // hóa với prange thay range
        bootstrappedDataFrame = bootstrapSample(dataFrame, bootstrapSize)
        decisionTree = buildDecisionTree(bootstrappedDataFrame, maxDepth = treeMaxDepth, randomAttributes = randomAttributes, randomSplits = randomSplits)
        forest.append(decisionTree)
    return forest



def randomForestPredictions(dataFrame, randomForest):
    predictions = {}
    for i in prange(len(randomForest)): # // hóa với prange thay range
        column = "decision tree " + str(i)
        predictions[column] = decisionTreePredictions(dataFrame, randomForest[i])
    predictions = pandas.DataFrame(predictions)
    return predictions.mode(axis = 1)[0]

def calculateAccuracy(predictedResults, category):
    resultCorrect = predictedResults == category
    return resultCorrect.mean()

## Sample run

In [ ]:
# Load the dataset
df = pd.read_csv("/content/drive/MyDrive/Bài Tập Về Nhà Môn Cuối/iris.csv")

%time
randomForest = createRandomForest(df, bootstrapSize = 10, randomAttributes = 10, randomSplits = 50, forestSize = 30, treeMaxDepth = 3)
randomForestTestResults = randomForestPredictions(df, randomForest)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.39 µs


In [ ]:
accuracy = calculateAccuracy(randomForestTestResults, df.iloc[:, -1]) * 100
accuracy

95.33333333333334

# 3. Song song hóa sử dụng CPU & GPU **[ V3 ]**

## Decision Tree V3.0

In [ ]:
import numpy
import pandas
import random
from numba import njit, prange, cuda, float32, int32
import numba



'''

Hàm bên dưới xác định "độ tinh khiết (Pure)" của tập dữ liệu bằng cách kiểm tra xem nó

có chứa cùng một label hay không, nếu có thì return true, ngược lại return false.

'''

def checkPurity(data):
    if len(numpy.unique(data[:, -1])) == 1:
        return True
    else:
        return False

    # ...



def classifyData(data):
    uniqueClasses, uniqueClassesCounts = numpy.unique(data[:, -1], return_counts = True)
    return uniqueClasses[uniqueClassesCounts.argmax()]

    # ...



def getPotentialSplits(data, randomAttributes):
    potentialSplits = {}
    _, columns = data.shape
    columnsIndices = list(range(columns - 1))
    if randomAttributes != None  and len(randomAttributes) <= len(columnsIndices):
        columnsIndices = randomAttributes
    for column in columnsIndices:
        values = data[:, column]
        uniqueValues = numpy.unique(values)
        if len(uniqueValues) == 1:
            potentialSplits[column] = uniqueValues
        else:
            potentialSplits[column] = []
            for i in prange(len(uniqueValues)): # // ở đây bằng prange
                if i != 0:
                    currentValue = uniqueValues[i]
                    previousValue = uniqueValues[i - 1]
                    potentialSplits[column].append((currentValue + previousValue) / 2) # tại sao dừng công thức này ?
    return potentialSplits



def splitData(data, splitColumn, splitValue):
    splitColumnValues = data[:, splitColumn]
    return data[splitColumnValues <= splitValue], data[splitColumnValues > splitValue]


# CUDA kernel ref. https://nyu-cds.github.io/python-numba/05-cuda/
@cuda.jit
def calculateEntropy(uniqueClassesCounts, lendata, result):

    idx = cuda.grid(1)  # Get the thread index


    if idx < lendata:
      # t, uniqueClassesCounts = np.unique(data[:, -1], return_counts = True)
      sum_uniqueClassesCounts = 0.0

      for i in uniqueClassesCounts:
        sum_uniqueClassesCounts += i

      # probabilities = uniqueClassesCounts / sum_uniqueClassesCounts


      # // ở đây, thay thế hàm sum có sẵn trong python bằng vòng lặp sử dụng prange
      # entropy = 0.0
      # for i in probabilities:
      #     entropy -= i * cuda.log2(i)

    result[idx] = 0.0

    # ...



def calculateOverallEntropy(dataBelow, dataAbove):

    pDataBelow = len(dataBelow) / (len(dataBelow) + len(dataAbove))
    pDataAbove = len(dataAbove) / (len(dataBelow) + len(dataAbove))

    # Allocate device memory for entropy calculation
    entropyBelow = cuda.to_device(len(dataBelow))
    entropyAbove = cuda.to_device(len(dataAbove))

    # calculate amout of unique class
    _, Below_uniqueClassesCounts = np.unique(dataBelow[:, -1], return_counts = True)
    _, Above_uniqueClassesCounts = np.unique(dataAbove[:, -1], return_counts = True)



    # Launch CUDA kernels for entropy calculation
    blockSize = 128
    gridSizeBelow = (len(dataBelow) + blockSize - 1) // blockSize
    gridSizeAbove = (len(dataAbove) + blockSize - 1) // blockSize

    print(f'{dataBelow=}')


    calculateEntropy[gridSizeBelow, blockSize](Below_uniqueClassesCounts, len(dataBelow), entropyBelow)
    calculateEntropy[gridSizeAbove, blockSize](Above_uniqueClassesCounts, len(dataAbove), entropyAbove)

    # Transfer results back to the host
    entropyBelow = entropyBelow.copy_to_host()
    entropyAbove = entropyAbove.copy_to_host()

    print(entropyBelow, '\n')
    print(entropyAbove)

    overallEntropy = pDataBelow * entropyBelow.sum() + pDataAbove * entropyAbove.sum()
    return overallEntropy

    # ...


def determineBestSplit(data, potentialSplits, randomSplits = None):
    overallEntropy = 9999
    bestSplitColumn = 0
    bestSplitValue = 0
    if randomSplits == None:
        for splitColumn in potentialSplits:
            for splitValue in potentialSplits[splitColumn]:
                dataBelow, dataAbove = splitData(data, splitColumn, splitValue)
                currentOverallEntropy = calculateOverallEntropy(dataBelow, dataAbove)
                if currentOverallEntropy <= overallEntropy:
                    overallEntropy = currentOverallEntropy
                    bestSplitColumn = splitColumn
                    bestSplitValue = splitValue
    else:
        for i in prange(randomSplits): # // vòng for này bằng prange
            randomSplitColumn = random.choice(list(potentialSplits))
            randomSplitValue = random.choice(potentialSplits[randomSplitColumn])
            dataBelow, dataAbove = splitData(data, randomSplitColumn, randomSplitValue)

            # print(f'{randomSplitValue=}\n{dataBelow=}\n {dataAbove=}\n\n')
            currentOverallEntropy = calculateOverallEntropy(dataBelow, dataAbove)
            if currentOverallEntropy <= overallEntropy:
                overallEntropy = currentOverallEntropy
                bestSplitColumn = randomSplitColumn
                bestSplitValue = randomSplitValue
    return bestSplitColumn, bestSplitValue



def buildDecisionTree(dataFrame, currentDepth = 0, minSampleSize = 2, maxDepth = 1000, randomAttributes = None, randomSplits = None):
    if currentDepth == 0:
        global COLUMN_HEADERS
        COLUMN_HEADERS = dataFrame.columns
        data = dataFrame.values
        if randomAttributes != None and randomAttributes <= len(COLUMN_HEADERS) - 1:
            randomAttributes = random.sample(population = list(range(len(COLUMN_HEADERS) - 1)), k = randomAttributes)
        else:
            randomAttributes = None
    else:
        data = dataFrame
    if checkPurity(data) or len(data) < minSampleSize or currentDepth == maxDepth:
        return classifyData(data)
    else:
        currentDepth += 1
        potentialSplits = getPotentialSplits(data, randomAttributes)
        splitColumn, splitValue = determineBestSplit(data, potentialSplits, randomSplits)
        dataBelow, dataAbove = splitData(data, splitColumn, splitValue)
        if len(dataBelow) == 0 or len(dataAbove) == 0:
            return classifyData(data)
        else:
            question = str(COLUMN_HEADERS[splitColumn]) + " <= " + str(splitValue)
            decisionSubTree = {question: []}
            yesAnswer = buildDecisionTree(dataBelow, currentDepth, minSampleSize, maxDepth, randomAttributes, randomSplits)
            noAnswer = buildDecisionTree(dataAbove, currentDepth, minSampleSize, maxDepth, randomAttributes, randomSplits)
            if yesAnswer == noAnswer:
                decisionSubTree = yesAnswer
            else:
                decisionSubTree[question].append(yesAnswer)
                decisionSubTree[question].append(noAnswer)
            return decisionSubTree



def classifySample(sample, decisionTree):
    if not isinstance(decisionTree, dict):
        return decisionTree
    question = list(decisionTree.keys())[0]
    attribute, value = question.split(" <= ")
    if sample[attribute] <= float(value):
        answer = decisionTree[question][0]
    else:
        answer = decisionTree[question][1]
    return classifySample(sample, answer)



def decisionTreePredictions(dataFrame, decisionTree):
    predictions = dataFrame.apply(classifySample, axis = 1, args = (decisionTree,))
    return predictions



def calculateAccuracy(predictedResults, category):
    resultCorrect = predictedResults == category
    return resultCorrect.mean()

## Random Forest V3.0

In [ ]:
def trainTestSplit(dataFrame, testSize):
    if isinstance(testSize, float):
        testSize = round(testSize * len(dataFrame))
    indices = dataFrame.index.tolist()
    testIndices = random.sample(population = indices, k = testSize)
    dataFrameTest = dataFrame.loc[testIndices]
    dataFrameTrain = dataFrame.drop(testIndices)
    return dataFrameTrain, dataFrameTest

def bootstrapSample(dataFrame, bootstrapSize):
    randomIndices = numpy.random.randint(low = 0, high = len(dataFrame), size = bootstrapSize)
    return dataFrame.iloc[randomIndices]



def createRandomForest(dataFrame, bootstrapSize, randomAttributes, randomSplits, forestSize = 20, treeMaxDepth = 1000):
    forest = []
    for i in prange(forestSize): # // hóa với prange thay range
        bootstrappedDataFrame = bootstrapSample(dataFrame, bootstrapSize)
        decisionTree = buildDecisionTree(bootstrappedDataFrame, maxDepth = treeMaxDepth, randomAttributes = randomAttributes, randomSplits = randomSplits)
        forest.append(decisionTree)
    return forest



def randomForestPredictions(dataFrame, randomForest):
    predictions = {}
    for i in prange(len(randomForest)): # // hóa với prange thay range
        column = "decision tree " + str(i)
        predictions[column] = decisionTreePredictions(dataFrame, randomForest[i])
    predictions = pandas.DataFrame(predictions)
    return predictions.mode(axis = 1)[0]

def calculateAccuracy(predictedResults, category):
    resultCorrect = predictedResults == category
    return resultCorrect.mean()

## Sample run

In [ ]:
# Load the dataset
df = pd.read_csv("/content/drive/MyDrive/Bài Tập Về Nhà Môn Cuối/iris.csv")

# label encoder
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()

# Encode labels in column 'species'.
df['species']= label_encoder.fit_transform(df['species'])

%time
randomForest = createRandomForest(df, bootstrapSize = 10, randomAttributes = 10, randomSplits = 50, forestSize = 30, treeMaxDepth = 3)
randomForestTestResults = randomForestPredictions(df, randomForest)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.15 µs
dataBelow=array([[4.4, 2.9, 1.4, 0.2, 0. ],
       [5.4, 3.4, 1.5, 0.4, 0. ],
       [5.8, 2.7, 3.9, 1.2, 1. ],
       [5.5, 2.4, 3.7, 1. , 1. ]])


TypingError: ignored

In [ ]:
accuracy = calculateAccuracy(randomForestTestResults, df.iloc[:, -1]) * 100
accuracy

In [ ]:
import numpy as np
import numba.cuda as cuda

# Define a CUDA kernel function using numba.cuda.jit
@cuda.jit
def square_array(arr):
    # Calculate the global index of the current thread
    idx = cuda.grid(1)

    # Perform the computation
    if idx < arr.size:
        arr[idx] = arr[idx] ** 2

# Create an input array
data = np.arange(10, dtype=np.float32)

# Transfer the input array to the GPU
d_data = cuda.to_device(data)

# Configure the kernel launch
threads_per_block = 64
blocks_per_grid = (data.size + (threads_per_block - 1)) // threads_per_block

# Launch the CUDA kernel
square_array[blocks_per_grid, threads_per_block](d_data)

# Transfer the modified array back to the host
result = d_data.copy_to_host()

# Print the result
print(result)


In [ ]:
import numpy as np
dt = np.array([[1, 2, 7],
               [3, 4, 8],
               [5, 6, 9],
               [11, 12, 13]])

psl = getPotentialSplits(dt, [0,1])



In [ ]:
t, uniqueClassesCounts = np.unique(dt[:, -1], return_counts = True)

In [ ]:
uniqueClassesCounts

array([1, 1, 1, 1])

In [ ]:
from numba import cuda

@cuda.reduce
def my_sum(a, b):
    return a + b

@cuda.jit
def my_cuda_kernel(data, res):
    i, j = cuda.grid(2)
    if i < data.shape[0] and j < data.shape[1]:
        # Perform calculations or assignments
        row_sum = my_sum(data[i, :])  # Calculate sum along rows
        col_sum = my_sum(data[:, j])  # Calculate sum along columns
        # Perform further computations or assignments with the row_sum and col_sum
    res[i, j] = row_sum


import numpy as np
from numba import cuda

# Generate sample data
# data = np.random.rand(10, 10)

res = 0

# Define CUDA kernel configuration
threads_per_block = (16, 16)
blocks_per_grid_x = (dt.shape[0] + threads_per_block[0] - 1) // threads_per_block[0]
blocks_per_grid_y = (dt.shape[1] + threads_per_block[1] - 1) // threads_per_block[1]
blocks_per_grid = (blocks_per_grid_x, blocks_per_grid_y)

# Allocate device memory for the data
d_data = cuda.to_device(dt)
d_res = cuda.to_device(res)

# Launch the CUDA kernel
my_cuda_kernel[blocks_per_grid, threads_per_block](d_data, d_res)

# Copy the results back to the host
result_data = d_data.copy_to_host()
res_res = d_res.copy_to_host()

print(result_data)


In [ ]:
import numpy as np
from numba import cuda

@cuda.jit
def sum_array_parallel(array, result):
    _,i = cuda.grid(2)
    if i < array.shape[1]:
        result[i] -= 1

# Generate sample data
data = dt

# Define CUDA kernel configuration
block_size = 8
num_blocks = (data.shape[0] + block_size - 1) // block_size

# Allocate device memory for the data and result
d_data = cuda.to_device(data)
result = np.array([0], dtype=np.int32)
d_result = cuda.to_device(result)

# Launch the CUDA kernel
sum_array_parallel[num_blocks, block_size](d_data, d_result)

# Copy the result back to the host
result = d_result.copy_to_host()

print(result[0])  # Output: 45 (sum of elements in the array)


-1


In [ ]:
psl

In [ ]:
determineBestSplit(dt, psl, None)